In [1]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
import PIL
from facenet_pytorch import MTCNN, InceptionResnetV1

In [2]:
mtcnn=MTCNN()
model=InceptionResnetV1(pretrained='vggface2').eval()

In [8]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [9]:
data=torchvision.datasets.ImageFolder("D:\\si\\Faces for Training\\Faces for Training",transform=transform)

In [10]:
device=torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [6]:
# model=torchvision.models.vgg16(pretrained=True)

In [7]:
# for i in model.parameters():
#     i.requires_grad=False

In [11]:
print(model)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [9]:
# for i in model.parameters():
#     i.requires_grad=False

In [10]:
class no(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x

In [11]:
model.avgpool=nn.AdaptiveAvgPool2d((1,1))
model.classifier=nn.Sequential(
    
        nn.Linear(512,256),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5,inplace=False),
        nn.Linear(256,128)
    
    )

In [12]:
net=model.to(device)

In [13]:

class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        super().__init__()
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)

       
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] == img1_tuple[1]:
                    break
        else:

            while True:
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] != img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])

        img0 = img0.convert("L")
        img1 = img1.convert("L")

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [18]:
sdata=SiameseNetworkDataset(data,transform)

In [19]:
data_loader=DataLoader(sdata,64,shuffle=True,num_workers=2)

In [ ]:
for i,l in data_loader:
    plt.imshow(i[0,0,:,:])
    print(l)
    break

In [20]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
       
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [21]:

criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0005 )

In [ ]:
counter = []
loss_history = [] 
iteration_number= 0

for epoch in range(10):


    for i, (img0, img1, label) in enumerate(data_loader, 0):

       
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()

        
        optimizer.zero_grad()

        
        output1, output2 = net(img0, img1)

        
        loss_contrastive = criterion(output1, output2, label)

       
        loss_contrastive.backward()

        
        optimizer.step()

        
        if i % 50 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 50

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())

